# LangGraph: State Machines с реальной LLM

**Цель:** граф состояний с локальной TinyLlama, жёсткими и условными рёбрами.

**Граф:**
```
retrieve → generate → [router] ─┬→ retry (retrieve)  [если ответ короткий]
                                └→ END               [иначе]
```

---

### 🎯 Якорь
LangGraph = карта маршрутов: жёсткие рёбра (всегда A→B) и условные (router решает: retry или конец).

In [ ]:
# ← 1. Установка (Colab: Runtime → GPU)
!pip install -q langgraph langchain langchain-core langchain-community
!pip install -q transformers accelerate torch

> ⚠️ **Если ошибка `torchvision::nms does not exist`:** используй `model.generate()` напрямую вместо pipeline — см. [ie_extraction_hw](09_hw_ie_extraction) как обойти импорт pipeline.

## 1. Загрузка малой LLM (TinyLlama 1.1B)

TinyLlama помещается в Colab T4 (~2–4 GB VRAM). Без 4-bit — быстрее для демо.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # открытая, ~2.2 GB

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)
print("LLM загружена, device:", next(model.parameters()).device)

## 2. State и узлы графа

- **State:** messages, context, retry_count
- **retrieve:** подставляет контекст (демо — фиксированный)
- **generate:** вызывает LLM по промпту с контекстом

In [ ]:
from typing import TypedDict
from langchain_core.messages import HumanMessage, AIMessage

# ← 1. Состояние графа
class State(TypedDict):
    messages: list
    context: str
    retry_count: int

# Демо-база "документов" (в реальности — retriever.invoke)
FAKE_DOCS = """RAG — это Retrieval-Augmented Generation. Система ищет релевантные документы по вопросу и подкладывает их в промпт для LLM. LLM отвечает на основе контекста, а не только своих знаний."""

def retrieve(state: State) -> dict:
    """Узел 1: «подтягиваем» контекст. В демо — фиксированный."""
    return {"context": FAKE_DOCS}

def generate(state: State) -> dict:
    """Узел 2: генерируем ответ по контексту и вопросу."""
    if TRACE:
        print(f"\n  [УЗЕЛ] generate ← по ребру retrieve→generate (жёсткое)")
    question = state["messages"][-1].content if state["messages"] else ""
    prompt = f"""Контекст: {state['context']}\nВопрос: {question}\nКраткий ответ:"""
    response = llm.invoke(prompt)
    return {"messages": state["messages"] + [AIMessage(content=response)]}

# Включить трассировку обхода графа (False = без логов)
TRACE = True

## 3. Роутер: условное ребро

Если ответ слишком короткий И retry_count < 2 → идём снова в retrieve (цикл). Иначе → END.

In [ ]:
def router(state: State) -> str:
    """Роутер: решаем, retry или конец."""
    last = state["messages"][-1]
    text = last.content if hasattr(last, "content") else str(last)
    rc = state.get("retry_count", 0)
    short = len(text.strip()) < 20
    # Короткий ответ (< 20 символов без пробела) — пробуем ещё раз
    if short and rc < 2:
        if TRACE:
            print(f"  [ВЕТВЛЕНИЕ] router: ответ={len(text.strip())} символов, retry_count={rc} → retry (цикл в retrieve)")
        return "retry"
    if TRACE:
        print(f"  [ВЕТВЛЕНИЕ] router: ответ={len(text.strip())} символов, retry_count={rc} → end (выход)")
    return "end"

#        ↑
#        └── ✨ ВОТ ЗДЕСЬ МАГИЯ: по state выбираем следующий узел

## 4. Сборка графа: узлы + рёбра

- Жёсткое ребро: `retrieve → generate`
- Условное ребро: `generate → [router] → retrieve | END`

In [ ]:
from langgraph.graph import StateGraph, END

# При retry увеличиваем счётчик (роутер ограничивает цикл)
def retrieve_with_count(state: State) -> dict:
    step = state.get("retry_count", 0) + 1
    if TRACE:
        src = "START" if step == 1 else "router(retry)"
        print(f"\n  [УЗЕЛ] retrieve ← по ребру {src}→retrieve (итерация {step})")
    out = retrieve(state)
    out["retry_count"] = step
    return out

workflow = StateGraph(State)

# Узлы
workflow.add_node("retrieve", retrieve_with_count)
workflow.add_node("generate", generate)

# Жёсткое ребро: retrieve → generate
workflow.add_edge("retrieve", "generate")

# Условное ребро: generate → router → retrieve или END
workflow.add_conditional_edges("generate", router, {"retry": "retrieve", "end": END})

# Стартовый узел
workflow.set_entry_point("retrieve")

app = workflow.compile()  # recursion_limit задаётся в config при invoke()
print("Граф скомпилирован. Рёбра: retrieve→generate, generate→[router]→retrieve|END")

## 5. Трассировка: как граф ходит по рёбрам

При `invoke()` ниже будет видно:
- **УЗЕЛ** — вход в узел и по какому ребру пришли
- **ВЕТВЛЕНИЕ** — решение роутера: `retry` (цикл) или `end` (выход)

## 6. Запуск

In [ ]:
print("=== Трассировка обхода графа ===")
result = app.invoke(
    {
        "messages": [HumanMessage(content="Что такое RAG?")],
        "context": "",
        "retry_count": 0,
    },
    config={"recursion_limit": 5},  # защита от бесконечного цикла
)

print("=== Результат ===")
for i, m in enumerate(result["messages"]):
    role = "User" if isinstance(m, HumanMessage) else "Assistant"
    print(f"[{role}] {m.content[:200]}{'...' if len(str(m.content)) > 200 else ''}")

print("\nRetry count:", result.get("retry_count", 0))

## Схема графа

```
     ┌──────────┐
     │  retrieve │  ← entry
     └─────┬─────┘
           │ add_edge (жёсткое)
           ↓
     ┌──────────┐
     │  generate │
     └─────┬─────┘
           │ add_conditional_edges
     router(state)
           │
     ┌─────┴─────┐
     ↓           ↓
 "retry"       "end"
     │           │
     ↓           ↓
 retrieve      END
 (цикл)
```